In [1]:
import sys
print(sys.executable)

C:\Users\ASUS\anaconda3\envs\mlflowcondaenv\python.exe


In [2]:
!python --version


Python 3.12.2


Without SMOTE

In [10]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [11]:
# Set up MLFlow to use a SQLite database
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("SVM Concentration Experiments")

<Experiment: artifact_location='file:///C:/Users/ASUS/e19-co544-Concentration-Difficulty-Prediction-Model-1/SVM/models_with_mlflow/mlruns/1', creation_time=1719295951403, experiment_id='1', last_update_time=1719295951403, lifecycle_stage='active', name='SVM Concentration Experiments', tags={}>

In [12]:
# Load your cleaned dataset
train_df = pd.read_csv("../../Preprocessing_pipeline/final_data_sets/train_set.csv")
train_df.head()

,Age,Sex,Relationship Status,Occupation,Social Media User?,Time Spent,Q1,Q2,Q3,Q4,...,Difficulty_level,Facebook,Instagram,YouTube,Snapchat,Twitter,Discord,Reddit,Pinterest,TikTok
0,24,0,1,2,1,2,3,3,2,2,...,0,1,1,1,0,1,0,0,0,0
1,46,0,2,2,1,0,2,1,1,1,...,0,1,0,1,0,0,0,0,0,0
2,30,1,2,2,1,4,2,4,4,2,...,1,1,1,0,0,0,0,0,0,0
3,56,1,2,3,1,1,1,1,1,1,...,0,0,0,1,0,0,0,0,0,0
4,19,0,1,1,1,4,3,3,4,3,...,1,1,1,1,0,0,0,0,0,1


In [13]:
test_df = pd.read_csv("../../Preprocessing_pipeline/final_data_sets/test_set.csv")
test_df.head()

,Age,Sex,Relationship Status,Occupation,Social Media User?,Time Spent,Q1,Q2,Q3,Q4,...,Difficulty_level,Facebook,Instagram,YouTube,Snapchat,Twitter,Discord,Reddit,Pinterest,TikTok
0,38,0,2,2,1,1,5,2,1,1,...,0,0,0,1,0,1,1,1,0,0
1,18,0,1,1,1,3,4,5,4,3,...,1,1,1,1,0,0,1,0,0,0
2,27,1,2,2,1,1,4,2,1,4,...,0,1,1,1,1,0,0,0,0,0
3,26,0,1,2,1,5,5,5,5,1,...,2,1,1,1,0,0,1,0,0,0
4,19,0,1,0,1,3,4,2,3,2,...,1,1,0,1,0,0,0,0,0,0


In [14]:
x_train = train_df.drop(columns=["Difficulty_level"])
y_train = train_df["Difficulty_level"]

In [15]:
x_test = test_df.drop(columns=["Difficulty_level"])
y_test = test_df["Difficulty_level"]

In [16]:
def train_and_log_model(kernel, C, gamma, feature_subset):
    # Select the subset of features
    X_train_subset = x_train[feature_subset]
    X_test_subset = x_test[feature_subset]

    # Initialize the SVM model with parameters
    model = SVC(kernel=kernel, C=C, gamma=gamma)

    # Create a unique run name
    run_name = f"SVM_{kernel}_C={C}_gamma={gamma}_features={'_'.join(feature_subset)}"

    # Print the run details for debugging
    print(f"Starting run: {run_name}")

    # Start an MLFlow run
    with mlflow.start_run(run_name=run_name):
        # Train the model
        model.fit(X_train_subset, y_train)

        # Make predictions
        y_pred = model.predict(X_test_subset)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted")
        recall = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        # Log parameters and metrics
        mlflow.log_param("kernel", kernel)
        mlflow.log_param("C", C)
        mlflow.log_param("gamma", gamma)
        mlflow.log_param("features", feature_subset)

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Log the model
        mlflow.sklearn.log_model(model, "model")

        # Print the metrics for debugging
        print(
            f"Logged run: {run_name} with accuracy={accuracy}, precision={precision}, recall={recall}, f1_score={f1}"
        )

In [17]:
kernels = ["linear", "rbf", "poly"]
C_values = [0.1, 1, 10]
gamma_values = ["scale", "auto"]
feature_combinations = [
    [
        "Age",
        "Sex",
        "Relationship Status",
        "Occupation",
        "Time Spent",
        "Social Media User?",
        "Q1",
        "Q2",
        "Q3",
        "Q4",
        "Q5",
        "Q6",
        "Q7",
        "Q8",
        "Facebook",
        "Instagram",
        "YouTube",
        "Snapchat",
        "Twitter",
        "Discord",
        "Reddit",
        "Pinterest",
        "TikTok",
    ],
    [
        "Age",
        "Relationship Status",
        "Occupation",
        "Time Spent",
        "Social Media User?",
        "Q1",
        "Q2",
        "Q3",
        "Q4",
        "Q5",
        "Q6",
        "Q7",
        "Facebook",
        "Instagram",
        "YouTube",
        "Snapchat",
        "Twitter",
        "Discord",
        "Reddit",
        "Pinterest",
        "TikTok",
    ],
    [
        "Age",
        "Sex",
        "Relationship Status",
        "Time Spent",
        "Social Media User?",
        "Q1",
        "Q2",
        "Q3",
        "Q4",
        "Q5",
        "Q6",
        "Q7",
        "Q8",
        "Facebook",
        "Instagram",
        "YouTube",
        "Snapchat",
        "Twitter",
        "Discord",
        "Reddit",
        "Pinterest",
        "TikTok",
    ],
    # Add more feature combinations as needed
]

In [18]:
for kernel in kernels:
    for C in C_values:
        for gamma in gamma_values:
            for feature_subset in feature_combinations:
                train_and_log_model(kernel, C, gamma, feature_subset)

print("Experiment complete. Check MLFlow UI for results.")

Starting run: SVM_linear_C=0.1_gamma=scale_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok


C:\Users\ASUS\anaconda3\envs\mlflowcondaenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logged run: SVM_linear_C=0.1_gamma=scale_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.7291666666666666, precision=0.6353383458646618, recall=0.7291666666666666, f1_score=0.6787941787941789
Starting run: SVM_linear_C=0.1_gamma=scale_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok


C:\Users\ASUS\anaconda3\envs\mlflowcondaenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logged run: SVM_linear_C=0.1_gamma=scale_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.7395833333333334, precision=0.6434294871794872, recall=0.7395833333333334, f1_score=0.6875
Starting run: SVM_linear_C=0.1_gamma=scale_features=Age_Sex_Relationship Status_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok


...

Logged run: SVM_poly_C=10_gamma=scale_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.7395833333333334, precision=0.64140625, recall=0.7395833333333334, f1_score=0.6847804190169219
Starting run: SVM_poly_C=10_gamma=scale_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok


C:\Users\ASUS\anaconda3\envs\mlflowcondaenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logged run: SVM_poly_C=10_gamma=scale_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.7395833333333334, precision=0.64140625, recall=0.7395833333333334, f1_score=0.6847804190169219
Starting run: SVM_poly_C=10_gamma=scale_features=Age_Sex_Relationship Status_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok


C:\Users\ASUS\anaconda3\envs\mlflowcondaenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logged run: SVM_poly_C=10_gamma=scale_features=Age_Sex_Relationship Status_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.7291666666666666, precision=0.6324460163812361, recall=0.7291666666666666, f1_score=0.6761083743842363
Starting run: SVM_poly_C=10_gamma=auto_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok
Logged run: SVM_poly_C=10_gamma=auto_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.4895833333333333, precision=0.5368303571428571, recall=0.4895833333333333, f1_score=0.5086358313817331
Starting run: SVM_poly_C=10_gamma=auto_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_

In [19]:
mlflow.set_experiment("SVM SMOTE Concentration Experiments")

2024/06/25 12:09:40 INFO mlflow.tracking.fluent: Experiment with name 'SVM SMOTE Concentration Experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/ASUS/e19-co544-Concentration-Difficulty-Prediction-Model-1/SVM/models_with_mlflow/mlruns/2', creation_time=1719297580322, experiment_id='2', last_update_time=1719297580322, lifecycle_stage='active', name='SVM SMOTE Concentration Experiments', tags={}>

In [20]:
# Load your cleaned dataset
train_df = pd.read_csv("../../Preprocessing_pipeline/final_data_sets/train_set_smote.csv")
train_df.head()

,Age,Sex,Relationship Status,Occupation,Social Media User?,Time Spent,Q1,Q2,Q3,Q4,...,Facebook,Instagram,YouTube,Snapchat,Twitter,Discord,Reddit,Pinterest,TikTok,Difficulty_level
0,24,0,1,2,1,2,3,3,2,2,...,1,1,1,0,1,0,0,0,0,0
1,46,0,2,2,1,0,2,1,1,1,...,1,0,1,0,0,0,0,0,0,0
2,30,1,2,2,1,4,2,4,4,2,...,1,1,0,0,0,0,0,0,0,1
3,56,1,2,3,1,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,0
4,19,0,1,1,1,4,3,3,4,3,...,1,1,1,0,0,0,0,0,1,1


In [21]:
x_train = train_df.drop(columns=["Difficulty_level"])
y_train = train_df["Difficulty_level"]

In [22]:
def train_and_log_model(kernel, C, gamma, feature_subset):
    # Select the subset of features
    X_train_subset = x_train[feature_subset]
    X_test_subset = x_test[feature_subset]

    # Initialize the SVM model with parameters
    model = SVC(kernel=kernel, C=C, gamma=gamma)

    # Create a unique run name
    run_name = f"SVM_{kernel}_C={C}_gamma={gamma}_features={'_'.join(feature_subset)}"

    # Print the run details for debugging
    print(f"Starting run: {run_name}")

    # Start an MLFlow run
    with mlflow.start_run(run_name=run_name):
        # Train the model
        model.fit(X_train_subset, y_train)

        # Make predictions
        y_pred = model.predict(X_test_subset)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted")
        recall = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        # Log parameters and metrics
        mlflow.log_param("kernel", kernel)
        mlflow.log_param("C", C)
        mlflow.log_param("gamma", gamma)
        mlflow.log_param("features", feature_subset)

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Log the model
        mlflow.sklearn.log_model(model, "model")

        # Print the metrics for debugging
        print(
            f"Logged run: {run_name} with accuracy={accuracy}, precision={precision}, recall={recall}, f1_score={f1}"
        )

In [23]:
kernels = ["linear", "rbf", "poly"]
C_values = [0.1, 1, 10]
gamma_values = ["scale", "auto"]
feature_combinations = [
    [
        "Age",
        "Sex",
        "Relationship Status",
        "Occupation",
        "Time Spent",
        "Social Media User?",
        "Q1",
        "Q2",
        "Q3",
        "Q4",
        "Q5",
        "Q6",
        "Q7",
        "Q8",
        "Facebook",
        "Instagram",
        "YouTube",
        "Snapchat",
        "Twitter",
        "Discord",
        "Reddit",
        "Pinterest",
        "TikTok",
    ],
    [
        "Age",
        "Relationship Status",
        "Occupation",
        "Time Spent",
        "Social Media User?",
        "Q1",
        "Q2",
        "Q3",
        "Q4",
        "Q5",
        "Q6",
        "Q7",
        "Facebook",
        "Instagram",
        "YouTube",
        "Snapchat",
        "Twitter",
        "Discord",
        "Reddit",
        "Pinterest",
        "TikTok",
    ],
    [
        "Age",
        "Sex",
        "Relationship Status",
        "Time Spent",
        "Social Media User?",
        "Q1",
        "Q2",
        "Q3",
        "Q4",
        "Q5",
        "Q6",
        "Q7",
        "Q8",
        "Facebook",
        "Instagram",
        "YouTube",
        "Snapchat",
        "Twitter",
        "Discord",
        "Reddit",
        "Pinterest",
        "TikTok",
    ],
    # Add more feature combinations as needed
]

In [24]:
for kernel in kernels:
    for C in C_values:
        for gamma in gamma_values:
            for feature_subset in feature_combinations:
                train_and_log_model(kernel, C, gamma, feature_subset)

print("Experiment complete. Check MLFlow UI for results.")

Starting run: SVM_linear_C=0.1_gamma=scale_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok
Logged run: SVM_linear_C=0.1_gamma=scale_features=Age_Sex_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Q8_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0.5416666666666666, precision=0.6515386928628709, recall=0.5416666666666666, f1_score=0.5661101876418502
Starting run: SVM_linear_C=0.1_gamma=scale_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok
Logged run: SVM_linear_C=0.1_gamma=scale_features=Age_Relationship Status_Occupation_Time Spent_Social Media User?_Q1_Q2_Q3_Q4_Q5_Q6_Q7_Facebook_Instagram_YouTube_Snapchat_Twitter_Discord_Reddit_Pinterest_TikTok with accuracy=0